# TEST QUALITE PREDICTION

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
np.random.seed(1)

paris_listings = pd.read_csv("paris_airbnb.csv")
paris_listings = paris_listings.loc[np.random.permutation(len(paris_listings))]
stripped_commas = paris_listings['price'].str.replace(",","")
stripped_dollars  = stripped_commas.str.replace("$","")
paris_listings['price'] = stripped_dollars.astype('float')
train_df = paris_listings.iloc[0:6000]
test_df = paris_listings.iloc[6000:]

def predict_price(new_listing):
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['accommodates'].apply(lambda x: np.abs(x - new_listing))
    temp_df = temp_df.sort_values('distance')
    nearest_neighbor_prices = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbor_prices.mean()
    return(predicted_price)

test_df['predicted_price'] = test_df['accommodates'].apply(lambda x: predict_price(x))

# LES METRIQUES D'ERREUR

In [2]:
test_df.head()

,host_response_rate,host_acceptance_rate,host_listings_count,latitude,longitude,city,zipcode,state,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,predicted_price
4465,96%,NaN,28.0,48.89318,2.31884,Paris,75017,Paris province,2,Entire home/apt,0.0,1.0,1.0,63.0,$100.00,$400.00,14,180,61,90.4
3154,100%,NaN,2.0,48.85114,2.29728,Paris,75015,Île-de-France,5,Entire home/apt,1.0,1.0,4.0,120.0,$50.00,$200.00,1,120,102,176.8
1969,65%,NaN,93.0,48.85134,2.33359,Paris,75006,Île-de-France,3,Entire home/apt,1.0,1.0,2.0,149.0,$60.00,$600.00,30,1125,5,96.2
2713,NaN,NaN,1.0,48.87320,2.38649,Paris,75020,Île-de-France,4,Entire home/apt,2.0,1.0,1.0,79.0,$65.00,$299.00,2,1125,5,173.0
5578,100%,NaN,1.0,48.85691,2.36013,Paris,75004,Île-de-France,2,Entire home/apt,1.0,1.0,1.0,51.0,$25.00,$150.00,4,15,17,90.4


In [3]:
test_df['error'] = np.absolute(test_df["predicted_price"] - test_df["price"])

In [7]:
mae = test_df['error'].mean()

In [9]:
mae

52.5621999999998

# ERREUR QUADRATIQUE MOYENNE

In [10]:
test_df['quadratic error']=(test_df['price']-test_df['predicted_price'])**2

In [11]:
mse = test_df['quadratic error'].mean()

In [12]:
mse

6724.726040000068

In [13]:
from math import *
rmse = sqrt(mse)
rmse

82.00442695367164

# ENTRAINER UN AUTRE MODELE

In [14]:
def predict_price(new_listing):
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['bedrooms'].apply(lambda x: np.abs(x - new_listing))
    temp_df = temp_df.sort_values('distance')
    nearest_neighbor_prices = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbor_prices.mean()
    return(predicted_price)

#test_df['predicted_price']=test.df['bedrooms'].apply(lambda x: predict_price(x))

In [15]:
test_df['predicted_price']=test_df['bedrooms'].apply(predict_price)

In [16]:
test_df['quadratic error']=(test_df['predicted_price']-test_df['price'])**2

In [17]:
mse = test_df['quadratic error'].mean()

In [19]:
print(mse)

6281.181980000002

# RACINE CARREE DE L'ERREUR QUADRATIQUE MOYENNE

In [20]:
from math import *

In [21]:
rmse=sqrt(mse)

In [22]:
rmse

79.25390829479643

# Comparaison des erreurs

In [78]:
errors_one = pd.Series([5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10])
errors_two = pd.Series([5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 1000])

In [94]:
mae_one = errors_one.mean()

In [136]:
s1 = np.mean(errors_one ** 2)
rmse_one = sqrt(s1)

In [137]:
rmse_one

7.905694150420948

In [122]:
mae_one

7.5

In [123]:
mae_two = errors_two.mean()

In [138]:
s2 = np.mean(errors_two**2)
rmse_two = sqrt(s2)

In [139]:
rmse_two

235.82302686548658

In [140]:
mae_two

62.5

In [129]:
s1


0      25
1     100
2      25
3     100
4      25
5     100
6      25
7     100
8      25
9     100
10     25
11    100
12     25
13    100
14     25
15    100
16     25
17    100
dtype: int64

In [133]:
s2

0          25
1         100
2          25
3         100
4          25
5         100
6          25
7         100
8          25
9         100
10         25
11        100
12         25
13        100
14         25
15        100
16         25
17    1000000
dtype: int64